In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class ConditionalNormalizingFlow(nn.Module):
    """
    Conditional Normalizing Flow for Density Estimation
    
    This implementation demonstrates a simple conditional normalizing flow 
    that can learn complex conditional distributions.
    """
    def __init__(self, condition_dim, target_dim, hidden_dims=[64, 64]):
        """
        Initialize the Conditional Normalizing Flow
        
        Args:
            condition_dim (int): Dimension of the conditioning variables
            target_dim (int): Dimension of the target variable to estimate
            hidden_dims (list): Hidden layer dimensions for the transformation networks
        """
        super().__init__()
        
        # Base distribution (standard normal)
        self.base_dist = torch.distributions.Normal(0, 1)
        
        # Transformation network (conditional affine flow)
        layers = []
        prev_dim = condition_dim + target_dim
        
        # Hidden layers
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU()
            ])
            prev_dim = hidden_dim
        
        # Output layers for scale and shift
        self.scale_net = nn.Sequential(
            *layers,
            nn.Linear(prev_dim, target_dim)
        )
        
        self.shift_net = nn.Sequential(
            *layers,
            nn.Linear(prev_dim, target_dim)
        )
    
    def forward(self, condition, target):
        """
        Compute the log probability of the target given the condition
        
        Args:
            condition (torch.Tensor): Conditioning variables
            target (torch.Tensor): Target variables
        
        Returns:
            torch.Tensor: Log probability of the target
        """
        # Concatenate condition and target
        inp = torch.cat([condition, target], dim=-1)
        
        # Compute scale and shift
        scale = torch.sigmoid(self.scale_net(inp)) + 1e-3  # Ensure positivity
        shift = self.shift_net(inp)
        
        # Transform the target
        z = (target - shift) / scale
        
        # Compute log probability
        log_prob = self.base_dist.log_prob(z).sum(-1)
        log_det = -torch.log(scale).sum(-1)
        
        return log_prob + log_det
    
    def sample(self, condition, num_samples=1):
        """
        Sample from the conditional distribution
        
        Args:
            condition (torch.Tensor): Conditioning variables
            num_samples (int): Number of samples to generate
        
        Returns:
            torch.Tensor: Samples from the conditional distribution
        """
        # Repeat condition for multiple samples
        condition = condition.repeat(num_samples, 1)
        
        # Sample from base distribution
        z = self.base_dist.sample((num_samples, condition.shape[1])).to(condition.device)
        
        # Concatenate condition and base samples
        inp = torch.cat([condition, z], dim=-1)
        
        # Compute scale and shift
        scale = torch.sigmoid(self.scale_net(inp)) + 1e-3
        shift = self.shift_net(inp)
        
        # Transform samples
        return z * scale + shift

def generate_conditional_data():
    """
    Generate synthetic data for demonstration
    
    Returns:
        tuple: Conditions, targets, and true underlying distribution
    """
    # Set random seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Generate data where target depends non-linearly on condition
    def true_distribution(x):
        return 0.3 * np.sin(x) + 0.7 * np.cos(2*x) + np.random.normal(0, 0.2, x.shape)
    
    # Generate conditions
    conditions = np.linspace(-np.pi, np.pi, 500)
    
    # Generate targets with noise
    targets = true_distribution(conditions)
    
    return (
        torch.FloatTensor(conditions).unsqueeze(-1),
        torch.FloatTensor(targets).unsqueeze(-1),
        true_distribution
    )

def train_conditional_flow(epochs=2000):
    """
    Train the conditional normalizing flow
    
    Args:
        epochs (int): Number of training epochs
    
    Returns:
        tuple: Trained model, conditions, targets
    """
    # Generate data
    conditions, targets, true_dist = generate_conditional_data()
    
    # Initialize model
    model = ConditionalNormalizingFlow(
        condition_dim=1, 
        target_dim=1
    )
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    # Training loop
    for epoch in range(epochs):
        # Zero grad
        optimizer.zero_grad()
        
        # Compute negative log-likelihood
        loss = -model(conditions, targets).mean()
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        # Print loss periodically
        if epoch % 200 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    
    return model, conditions, targets


 

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
def visualize_results(model, conditions, targets, true_dist, num_samples_per_condition=100):
    """
    Visualize the learned conditional distribution with optimized memory usage.
    
    Args:
        model (ConditionalNormalizingFlow): Trained model
        conditions (torch.Tensor): Condition values
        targets (torch.Tensor): Target values
        true_dist (function): True underlying distribution
        num_samples_per_condition (int): Number of samples per condition to generate
    """
    plt.figure(figsize=(12, 5))
    
    # Test conditions for visualization
    test_conditions = torch.linspace(-np.pi, np.pi, 200).unsqueeze(-1)
    
    # Generate samples for different conditions
    samples_means = []
    for cond in test_conditions:
        with torch.no_grad():
            samples = model.sample(cond, num_samples=num_samples_per_condition)
        samples_means.append(samples.mean().item())
    
    # Convert test_conditions to numpy
    test_conditions_np = test_conditions.numpy().squeeze()
    
    # Plot 1: Learned vs True Means
    plt.subplot(1, 2, 1)
    plt.title('Learned vs True Distribution (Mean)')
    true_samples_means = [true_dist(x) for x in test_conditions_np]
    plt.plot(test_conditions_np, true_samples_means, label="True Distribution", color="red")
    plt.plot(test_conditions_np, samples_means, label="Learned Distribution", color="blue")
    plt.xlabel("Condition")
    plt.ylabel("Mean Target")
    plt.legend()
    
    # Plot 2: Sample Comparison
    plt.subplot(1, 2, 2)
    plt.title('Samples vs True Distribution')
    true_samples = [true_dist(x) for x in test_conditions_np]
    plt.scatter(
        test_conditions_np, 
        true_samples, 
        alpha=0.5, 
        label='True Distribution',
        color='red'
    )
    plt.scatter(
        test_conditions_np, 
        samples_means, 
        alpha=0.5, 
        label='Learned Distribution',
        color='blue'
    )
    
    plt.xlabel("Condition")
    plt.ylabel("Target")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Generate data as in the example
n_samples = 20000
y = np.random.uniform(-.5, 2.5, n_samples)
X = y.reshape(-1, 1)  # Reshape y into a 2D array (n_samples, 1)
xtrue = np.zeros_like(y)

# Apply conditions to generate xtrue based on y
for i in range(len(xtrue)):
    if (0 < y[i] < 0.5) or (1.5 < y[i] < 2):
        xtrue[i] = np.random.normal(1, scale=0.5)
    else:
        xtrue[i] = np.random.normal(-1, scale=0.5)

# Generate varying observation noise
obs_noise_std = np.random.uniform(0.5, 1.5, len(xtrue))
x = xtrue + np.random.normal(0, obs_noise_std, len(xtrue))

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
x = torch.tensor(x, dtype=torch.float32).reshape(-1, 1)

# Conditional Normalizing Flow Layer
class ConditionalCouplingLayer(nn.Module):
    def __init__(self, condition_dim, data_dim, hidden_dims=[64, 64]):
        super().__init__()
        
        # Total input dimension (condition + data)
        total_dim = condition_dim + data_dim
        
        # Scale network
        scale_layers = []
        prev_dim = total_dim
        for h_dim in hidden_dims:
            scale_layers.append(nn.Linear(prev_dim, h_dim))
            scale_layers.append(nn.ReLU())
            prev_dim = h_dim
        scale_layers.append(nn.Linear(prev_dim, data_dim))
        self.scale_net = nn.Sequential(*scale_layers)
        
        # Shift network
        shift_layers = []
        prev_dim = total_dim
        for h_dim in hidden_dims:
            shift_layers.append(nn.Linear(prev_dim, h_dim))
            shift_layers.append(nn.ReLU())
            prev_dim = h_dim
        shift_layers.append(nn.Linear(prev_dim, data_dim))
        self.shift_net = nn.Sequential(*shift_layers)
    
    def forward(self, condition, x, inverse=False):
        # Ensure condition and x have the same first dimension
        if condition.size(0) != x.size(0):
            # Repeat condition to match x's batch size
            condition = condition.repeat(x.size(0), 1)
        
        # Combine condition and data
        z = torch.cat([condition, x], dim=1)
        
        # Compute scale and shift
        scale = torch.sigmoid(self.scale_net(z) + 1)  # Ensure positive scale
        shift = self.shift_net(z)
        
        if not inverse:
            # Forward transformation
            x_transformed = x * scale + shift
            log_det = torch.log(scale).sum(dim=1)
            return x_transformed, log_det
        else:
            # Inverse transformation
            x_inv = (x - shift) / scale
            log_det = -torch.log(scale).sum(dim=1)
            return x_inv, log_det

# Conditional Normalizing Flow Model
class ConditionalDensityEstimator(nn.Module):
    def __init__(self, condition_dim, data_dim, n_flows=3, hidden_dims=[64, 64]):
        super().__init__()
        
        # Create multiple flow layers
        self.flows = nn.ModuleList([
            ConditionalCouplingLayer(condition_dim, data_dim, hidden_dims) 
            for _ in range(n_flows)
        ])
        
        # Base distribution (standard normal)
        self.base_dist = torch.distributions.Normal(
            torch.zeros(data_dim), 
            torch.ones(data_dim)
        )
    
    def forward(self, condition, x):
        # Initial log probability from base distribution
        z = x
        log_prob = self.base_dist.log_prob(z).sum(dim=1)
        
        # Apply transformations
        for flow in self.flows:
            z, log_det = flow(condition, z)
            log_prob += log_det
        
        return log_prob
    
    def sample(self, condition, num_samples=1):
        # Sample from base distribution
        z = self.base_dist.sample((num_samples,))
        
        # If condition is a single point, repeat it to match sample batch size
        if condition.size(0) == 1:
            condition = condition.repeat(num_samples, 1)
        
        # Apply inverse transformations
        for flow in reversed(self.flows):
            z, _ = flow(condition, z, inverse=True)
        
        return z

# Prepare data
X_train = X
x_train = x

# Instantiate the model
model = ConditionalDensityEstimator(
    condition_dim=1,  # y dimension
    data_dim=1        # x dimension
)

# Loss function (negative log-likelihood)
def loss_fn(model, X, x):
    log_prob = model(X, x)
    return -log_prob.mean()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 200
batch_size = 256

for epoch in range(n_epochs):
    # Shuffle data
    indices = torch.randperm(len(X_train))
    X_shuffled = X_train[indices]
    x_shuffled = x_train[indices]
    
    # Mini-batch training
    for i in range(0, len(X_train), batch_size):
        batch_X = X_shuffled[i:i+batch_size]
        batch_x = x_shuffled[i:i+batch_size]
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Compute loss
        loss = loss_fn(model, batch_X, batch_x)
        
        # Backpropagation
        loss.backward()
        
        # Optimize
        optimizer.step()
    
    # Print loss periodically
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Visualization of conditional distributions
plt.figure(figsize=(15, 5))

# Different conditions to sample from
conditions = [0.25, 1.0, 1.75]

for i, cond in enumerate(conditions, 1):
    # Generate samples for this condition
    cond_tensor = torch.tensor([[cond]], dtype=torch.float32)
    
    # Generate multiple samples
    samples = model.sample(cond_tensor, num_samples=1000).detach().numpy()
    
    plt.subplot(1, 3, i)
    sns.histplot(samples.flatten(), kde=True)
    plt.title(f'Conditional Distribution at y = {cond}')
    plt.xlabel('x')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()

# Print some diagnostics
print("\nModel Training Completed.")
print("Conditions tested:", conditions)

In [ ]:
model, conditions, targets = train_conditional_flow()

In [ ]:
_, _, true_dist = generate_conditional_data()

In [ ]:
visualize_results(model, conditions, targets, true_dist)
